In [34]:
import pandas as pd
import numpy as np
df_volume = pd.read_csv("trans_sales_ (1).csv")
df_volume

,customer,product,2004_qty_month1,2004_qty_month2,2004_qty_month3,2004_qty_month4,2004_qty_month5,2004_qty_month6,2004_qty_month7,2004_qty_month8,2004_qty_month9,2004_qty_month10,2004_qty_month11,2004_qty_month12,2004_qty_quarter1,2004_qty_quarter2,2004_qty_quarter3,2004_qty_quarter4,2004_Total_qty
0,T133820637,PRD1,0,0,0,0,30,0,0,0,0,30,0,0,0,30,0,30,60
1,T147810859,PRD2,0,10,0,0,0,0,0,0,0,0,0,0,10,0,0,0,10
2,T172860382,PRD2,0,0,10,0,0,30,0,0,0,0,0,0,10,30,0,0,40
3,41206,PRD2,126,63,123,66,33,0,113,60,175,0,0,0,312,99,348,0,759
4,41206,PRD3,0,0,110,20,7,90,0,30,179,90,30,150,110,117,209,270,706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,210176046,PRD2,1186,1054,930,1728,1060,870,820,960,480,0,0,0,3170,3658,2260,0,9088
1995,210176046,PRD3,1898,1568,1869,1395,1241,1484,1735,1569,2263,844,1520,1140,5335,4120,5567,3504,18526
1996,210176046,PRD1,512,270,284,500,604,339,401,238,478,540,580,480,1066,1443,1117,1600,5226
1997,210177012,PRD2,0,0,0,0,0,0,0,30,0,0,0,0,0,0,30,0,30


In [35]:
df_volume["product"].unique()

array(['PRD1', 'PRD2', 'PRD3'], dtype=object)

In [36]:
df_volume['customer'].value_counts()

50680088      3
27587         3
32254         3
32143         3
70286046      3
             ..
T161750073    1
110387057     1
T161780492    1
T161790699    1
T133820637    1
Name: customer, Length: 1025, dtype: int64

In [37]:
month_list = ['customer','product']
quarter_list = ['customer','product']
df_volume[month_list[0]] = df_volume[month_list[0]].astype("str")
df_volume[month_list[1]] = df_volume[month_list[1]].astype("str")

In [38]:
for cols in df_volume.columns:
    if 'month' in cols:
        month_list.append(cols)
        # print(month_list)
    if 'quarter' in cols:
        quarter_list.append(cols)
         # print(quarter_list)
# df_volume = df_volume[df_volume[col_list]]
# print("Month list: ",month_list)
# print("Quarter list: ",quarter_list)


In [39]:
volume_dfs = dict()

volume_dfs['df_volume_month'] = df_volume[month_list]
volume_dfs['df_volume_quarter'] = df_volume[quarter_list]
# print(df_volume_month.head())
# print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
# print(df_volume_quarter.head())
def volume_segmentation_rule(val):
    val1 = list(map(int,val.values.tolist()))
    min_range = sum(val1)//len(val1) -10
    max_range = sum(val1)//len(val1) +10
    res = all(ele >= min_range and ele < max_range for ele in val1)
#     return val1
    if val1 == sorted(val1) and val1.count(0)<2 and len(set(val1))!= 1 and len(set(val1))== len(val1):
        return "Incresing Trend"
    elif val1 == sorted(val1,reverse=True) and val1.count(0)<2 and len(set(val1))!= 1 and len(set(val1))== len(val1):
        return "Declining Trend"
    elif res == True:
        return "Flat Trend"
    else:
        return "Fluctuating Trend"
        

In [40]:
products = 'PRD1,PRD2,PRD3'.split(",")
products_dfs_dict = dict()
for prd in products:
    if prd in df_volume["product"].unique():
        for df_name,df_value in volume_dfs.items():
            products_dfs_dict[df_name+"_"+prd] = pd.DataFrame()
            products_dfs_dict[df_name+"_"+prd] = df_value[df_value['product']==prd]
            
            temp_df = pd.DataFrame()
            temp_df['segments'] = df_value.iloc[:,2:].apply(lambda val: volume_segmentation_rule(val),axis = 1)
            
            products_dfs_dict[df_name+"_"+prd].insert(loc = 2,column='segments',value=temp_df['segments'])

In [41]:
products_dfs_dict.keys()

dict_keys(['df_volume_month_PRD1', 'df_volume_quarter_PRD1', 'df_volume_month_PRD2', 'df_volume_quarter_PRD2', 'df_volume_month_PRD3', 'df_volume_quarter_PRD3'])

In [42]:
from threading import Thread
from time import sleep

In [46]:
def printing_dfs(dic1,tp):
    for i,j in dic1.items():
        if tp in i:
            print("DataFrame name is: ",i)
            print("****************************************")
            display(dic1[i].head(20))
            display(dic1[i]['segments'].value_counts())
            print()
            print()
#             return dic1[i].head()

In [47]:
t1 = Thread(target=printing_dfs,args=(products_dfs_dict,'quarter'))
t2 = Thread(target=printing_dfs,args=(products_dfs_dict,'month'))

t1.start()
t1.join()
sleep(1)
t2.start()
t2.join()



DataFrame name is:  df_volume_quarter_PRD1
****************************************


,customer,product,segments,2004_qty_quarter1,2004_qty_quarter2,2004_qty_quarter3,2004_qty_quarter4
0,T133820637,PRD1,Fluctuating Trend,0,30,0,30
5,41206,PRD1,Fluctuating Trend,900,630,859,499
7,30183075,PRD1,Fluctuating Trend,0,0,0,30
10,50678040,PRD1,Fluctuating Trend,140,0,0,0
13,51483171,PRD1,Fluctuating Trend,1650,2800,2020,1459
14,100277061,PRD1,Declining Trend,820,330,190,0
16,110482032,PRD1,Flat Trend,0,0,0,0
19,230170109,PRD1,Fluctuating Trend,0,0,0,90
20,300664045,PRD1,Flat Trend,0,0,0,0
21,360157013,PRD1,Fluctuating Trend,0,90,0,0


Fluctuating Trend    539
Flat Trend           200
Declining Trend       32
Incresing Trend       29
Name: segments, dtype: int64



DataFrame name is:  df_volume_quarter_PRD2
****************************************


,customer,product,segments,2004_qty_quarter1,2004_qty_quarter2,2004_qty_quarter3,2004_qty_quarter4
1,T147810859,PRD2,Flat Trend,10,0,0,0
2,T172860382,PRD2,Fluctuating Trend,10,30,0,0
3,41206,PRD2,Fluctuating Trend,312,99,348,0
8,50678040,PRD2,Fluctuating Trend,30,30,60,0
11,51483171,PRD2,Fluctuating Trend,704,439,740,0
15,110482032,PRD2,Fluctuating Trend,210,90,120,0
17,190280073,PRD2,Fluctuating Trend,150,62,0,0
18,230170109,PRD2,Fluctuating Trend,90,180,60,0
22,384164072,PRD2,Fluctuating Trend,114,265,267,0
25,410262012,PRD2,Fluctuating Trend,180,150,230,0


...

,customer,product,segments,2004_qty_month1,2004_qty_month2,2004_qty_month3,2004_qty_month4,2004_qty_month5,2004_qty_month6,2004_qty_month7,2004_qty_month8,2004_qty_month9,2004_qty_month10,2004_qty_month11,2004_qty_month12
1,T147810859,PRD2,Fluctuating Trend,0,10,0,0,0,0,0,0,0,0,0,0
2,T172860382,PRD2,Fluctuating Trend,0,0,10,0,0,30,0,0,0,0,0,0
3,41206,PRD2,Fluctuating Trend,126,63,123,66,33,0,113,60,175,0,0,0
8,50678040,PRD2,Fluctuating Trend,30,0,0,0,0,30,60,0,0,0,0,0
11,51483171,PRD2,Fluctuating Trend,330,234,140,164,165,110,265,155,320,0,0,0
15,110482032,PRD2,Fluctuating Trend,60,0,150,90,0,0,90,0,30,0,0,0
17,190280073,PRD2,Fluctuating Trend,58,0,92,62,0,0,0,0,0,0,0,0
18,230170109,PRD2,Fluctuating Trend,30,0,60,60,60,60,0,60,0,0,0,0
22,384164072,PRD2,Fluctuating Trend,40,34,40,47,90,128,160,87,20,0,0,0
25,410262012,PRD2,Fluctuating Trend,60,30,90,60,60,30,90,40,100,0,0,0


Fluctuating Trend    645
Flat Trend            27
Name: segments, dtype: int64



DataFrame name is:  df_volume_month_PRD3
****************************************


,customer,product,segments,2004_qty_month1,2004_qty_month2,2004_qty_month3,2004_qty_month4,2004_qty_month5,2004_qty_month6,2004_qty_month7,2004_qty_month8,2004_qty_month9,2004_qty_month10,2004_qty_month11,2004_qty_month12
4,41206,PRD3,Fluctuating Trend,0,0,110,20,7,90,0,30,179,90,30,150
6,30183075,PRD3,Fluctuating Trend,0,30,0,60,0,60,60,120,0,30,0,0
9,50678040,PRD3,Fluctuating Trend,80,30,0,60,0,0,0,0,0,0,0,0
12,51483171,PRD3,Fluctuating Trend,321,600,90,180,320,80,390,60,430,680,140,70
23,384164072,PRD3,Flat Trend,0,0,0,0,0,0,0,0,0,0,0,0
26,410262012,PRD3,Fluctuating Trend,60,30,60,0,120,0,30,70,60,30,60,30
29,411384110,PRD3,Fluctuating Trend,104,174,300,274,261,148,165,250,220,240,57,90
33,480483021,PRD3,Fluctuating Trend,0,0,0,0,240,0,0,30,0,0,0,0
37,4951161001,PRD3,Fluctuating Trend,30,0,0,0,0,0,0,0,0,0,0,0
40,T165910178,PRD3,Fluctuating Trend,0,0,0,0,0,0,0,0,0,0,10,0


Fluctuating Trend    503
Flat Trend            24
Name: segments, dtype: int64